In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [21]:
import pandas as pd
import numpy as np
# model_path = 'model.sav'
data_path = './data.csv'
# Read file from csv file
data_df = pd.read_csv(data_path)

def string_to_float(data, feature):
    l = data[feature].unique()
    l_dict = dict(zip(l, np.arange(0.0, len(l)+1)))
    return l_dict

def preprocess(d):
    clean_language = string_to_float(d,'language')
    clean_license = string_to_float(d,'license')
    d = d.drop(columns=['private', 'url', 'Unnamed: 0'])
    d['language'] = d['language'].apply(lambda x: clean_language[x])
    d['license'] = d['license'].apply(lambda x: clean_license[x])
    #d = d.drop(columns=['watchers'])

    X = d.drop(columns=['stars']).values
    y = d['stars'].values

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # X_train = torch.FloatTensor(X_train)
    # X_val = torch.FloatTensor(X_val)
    # y_train = torch.FloatTensor(y_train)
    # y_val = torch.FloatTensor(y_val)

    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = preprocess(data_df)

In [28]:
param_grid = {"n_estimators":[800,1000],"max_depth":[15,18,20]}
grid_search = GridSearchCV(RandomForestRegressor(),param_grid,scoring='neg_mean_squared_error',cv = 10)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [15, 18, 20],
                         'n_estimators': [500, 800, 1000]},
             scoring='neg_mean_squared_error')

In [29]:
predictions = grid_search.predict(X_val)
r2_ss = r2_score(y_val, predictions)
print(r2_ss)

0.9810384779485142


In [31]:
grid_search.best_params_

{'max_depth': 20, 'n_estimators': 800}

In [32]:
best_rfr = grid_search.best_estimator_

In [33]:
pred2 = best_rfr.predict(X_val)
print(r2_score(y_val,pred2))

0.9810384779485142


In [36]:
import pickle

model_name = 'model.pkl'
pickle.dump(best_rfr, open(model_name, 'wb'))


In [37]:
load_mdl = pickle.load(open(model_name, 'rb'))
print(r2_score(y_val, load_mdl.predict(X_val)))

0.9810384779485142
